In [1]:
# 00 - Setup Spark + Parquet (verificacoes rapidas)
# Execute cada celula na ordem. Este notebook nao processa dados; so garante o ambiente pronto.

In [ ]:

# Garantir acesso aos modulos do projeto (../)
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
print(f"PROJECT_ROOT adicionado ao sys.path: {PROJECT_ROOT}")


In [2]:
import sys, os, shutil, subprocess
from pathlib import Path

print(f"Python: {sys.version}")
print(f"Working dir: {Path.cwd()}")
print(f"Env PATH has Java? {'java' in os.environ.get('PATH','').lower()}")

# Checar Java instalado
java = shutil.which('java')
if java:
    try:
        out = subprocess.check_output([java, '-version'], stderr=subprocess.STDOUT, text=True)
        print(out)
    except Exception as exc:  # noqa: BLE001
        print(f"Aviso: falha ao ler java -version: {exc}")
else:
    print('Java nao encontrado no PATH. Instale Java 8+ e reabra o terminal.')

Python: 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
Working dir: /home/jovyan/work/notebooks
Env PATH has Java? False
openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)



In [3]:
# Definir credenciais do Kaggle aqui (opcional). Preencha e execute para criar ~/.kaggle/kaggle.json.
import json
from pathlib import Path

KAGGLE_USERNAME = 'marcowandraski'  # coloque aqui o username
KAGGLE_KEY = '2c5901cf78547b038c2e88f0828128e1'       # coloque aqui a key

if KAGGLE_USERNAME and KAGGLE_KEY:
    kaggle_dir = Path.home() / '.kaggle'
    kaggle_dir.mkdir(parents=True, exist_ok=True)
    cred_path = kaggle_dir / 'kaggle.json'
    cred_path.write_text(json.dumps({'username': KAGGLE_USERNAME, 'key': KAGGLE_KEY}))
    try:
        cred_path.chmod(0o600)
    except Exception:
        pass
    print(f'kaggle.json criado em {cred_path}')
else:
    print('KAGGLE_USERNAME/KAGGLE_KEY vazios. Preencha e reexecute esta celula se precisar baixar dados do Kaggle.')

kaggle.json criado em /home/jovyan/.kaggle/kaggle.json


In [4]:
# Instalar dependencias (se ainda nao instalou no ambiente)
# Comente se ja fez: pip install -r requirements.txt
import sys, subprocess

try:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'])
    print('Dependencias instaladas/atualizadas.')
except Exception as exc:  # noqa: BLE001
    print(f'Aviso: pip install falhou (talvez ja instaladas?): {exc}')

Aviso: pip install falhou (talvez ja instaladas?): Command '['/opt/conda/bin/python', '-m', 'pip', 'install', '-r', 'requirements.txt']' returned non-zero exit status 1.


In [5]:
# Verificar kaggle credenciais (necessario para notebooks Bronze)
from pathlib import Path
kaggle_json = Path.home() / '.kaggle' / 'kaggle.json'
if kaggle_json.exists():
    print(f'Credenciais Kaggle encontradas em {kaggle_json}')
else:
    print('Credenciais Kaggle NAO encontradas. Adicione kaggle.json em ~/.kaggle ou preencha a celula de credenciais acima.')

Credenciais Kaggle encontradas em /home/jovyan/.kaggle/kaggle.json


In [6]:
# Criar diretorios de dados
from spark_jobs.config import ensure_data_dirs, BRONZE_PATH, SILVER_PATH, GOLD_DIR
ensure_data_dirs()
print('Diretorios criados/verificados:')
print(f' - Bronze: {BRONZE_PATH}')
print(f' - Silver: {SILVER_PATH}')
print(f' - Gold dir: {GOLD_DIR}')

ModuleNotFoundError: No module named 'spark_jobs'

In [7]:
# Testar criacao de SparkSession e escrita/ leitura de parquet temporario
from spark_jobs.spark_session_manager import get_spark_session
from pyspark.sql import functions as F
from pyspark.sql import Row
from pathlib import Path

spark = get_spark_session('Smoke Test')

tmp_path = Path('data/_tmp_spark_check.parquet')

# Dataset pequeno para validar escrita/leitura
sample = [Row(id=i, value=f'v{i}') for i in range(5)]
df = spark.createDataFrame(sample)
df = df.withColumn('ts', F.current_timestamp())

print('Escrevendo parquet temporario...')
df.write.mode('overwrite').parquet(str(tmp_path))
print(f'Parquet escrito em {tmp_path}')

print('Lendo parquet temporario...')
df2 = spark.read.parquet(str(tmp_path))
df2.show(truncate=False)

print('OK: Spark leu/escreveu Parquet.')

ModuleNotFoundError: No module named 'spark_jobs'

In [8]:
# Limpar arquivo temporario (opcional)
try:
    tmp_path = Path('data/_tmp_spark_check.parquet')
    if tmp_path.exists():
        import shutil
        shutil.rmtree(tmp_path)
        print(f'Removido {tmp_path}')
except Exception as exc:  # noqa: BLE001
    print(f'Aviso: nao foi possivel remover tmp parquet: {exc}')

spark.stop()
print('Setup concluido.')

NameError: name 'spark' is not defined